In [ ]:
import netgen.gui
from ngsolve import *
import numpy as np
from scipy import random
from math import sqrt

In [ ]:
from netgen.geom2d import unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))
Draw (mesh)


# Poisson euqation

\begin{equation}
\int{\nabla u \ \nabla v dx} = \int{f \ v dx} 
\end{equation}


https://ngsolve.org/docu/nightly/i-tutorials/unit-2.2-eigenvalues/pinvit.html

In [ ]:
# Poisson problem with hom. Dirichlet BC
fes = H1(mesh, order=3, dirichlet=".*")
gfu_a = GridFunction(fes,name = "u_a_jac")
gfu_b = GridFunction(fes,name = "u_b_jac")
gfu_a_dir = GridFunction(fes,name = "u_a_dir")
gfu_b_dir = GridFunction(fes,name = "u_b_dir")
print(len(gfu_a.vec))

In [ ]:
def SolveCG(A, C, u_grid, f_grid, fes=fes, maxsteps:int=200, tol:float=1e-8, printrate=False, printLast=True, errConv=True):
    """My CG
    
    args:
    A... matrix of the BLF
    C... (inverse) matrix of the Preconditioner (NGSolve stores only the inverse of C)
    u_grid... vector of the solution gridfunction
    f_grid... vector of the RHS (LF)
    maxsteps... maximum number of iterations
    tol... (error) tolerance
    printrate... enables printing of iteration number and error per step
    """
    # Help vectors
    r = u_grid.CreateVector()
    d = u_grid.CreateVector()
    p = u_grid.CreateVector()

    # Temps
    dTp = u_grid.CreateVector()
    Ap = u_grid.CreateVector()
    Cd = u_grid.CreateVector()
    Cd_bef = u_grid.CreateVector()
    d_bef = u_grid.CreateVector()
    tmp = u_grid.CreateVector()
    #total_err = u_grid.CreateVector()
    # Next, we pick a random initial vector, which is zeroed on the Dirichlet boundary.
    r.FV().NumPy()[:] = random.rand(fes.ndof) # FV...flatvector --> linear memory chunk which provides numpy view
    u_grid.data = Projector(fes.FreeDofs(), True) * r # The projector clears the entries at the Dirichlet boundary
    f_grid.data = Projector(fes.FreeDofs(), True) * f_grid
    del r
    
    tmp.data = f_grid
    tmp.data -= A * u_grid
    d.data = tmp
    
    Cd.data = C * d
    p.data = Cd
    
    last_err = 0.
    total_err = 0.
    err_counter = 0
    """
    for k in range(maxsteps):
        dp = d * p
        Ap.data = 
        pAp = InnerProduct(p, )
    """
    for k in range(maxsteps):
        Ap.data = A * p
        pTAp = InnerProduct(p, Ap)
        assert(isinstance(pTAp, float))

        # Rayleigh
        #alpha = InnerProduct(d, p) / pTAp
        alpha = InnerProduct(Cd, d)
        alpha /= pTAp
        #print(alpha)

        # Update u
        tmp.data = u_grid
        tmp.data += alpha * p
        u_grid.data = tmp
        u_grid.data = Projector(fes.FreeDofs(), True) * u_grid
        
        # residual
        d_bef.data = d
        Cd_bef.data = Cd
        tmp.data = d 
        tmp.data -= alpha * Ap
        d.data = tmp
        # check boundaries of u_grid
        #for i in range(u_grid.space.ndofs)
        last_err = total_err
        total_err = InnerProduct(d, d)
        
        #if k == 500:
         #   print(d)
        if printrate:
            print(f"(step: {k} ,error: {total_err})")
        if total_err < tol**2: # if errortolerance reached
            print(f"Err tol reached: error={total_err}, tol={tol}")
            break
        if errConv and np.isclose(last_err, total_err, rtol=tol): # if "no" change in error
            if err_counter:
                print(f"Error did not improve: error={last_err}\n...terminating")
                break
            err_counter +=1

        # BETA
        Cd.data = C * d # C is actually the inverse: C^-1...NGSolve only stores C^-1
        beta = InnerProduct(d, Cd)
        dCd_bef = InnerProduct(d_bef, Cd_bef)
        beta /= dCd_bef
        #beta = InnerProduct(dC, d.data) / dTp
        
        # Update p
        tmp.data = Cd
        tmp.data -= beta * p
        p.data = tmp.data
    if printLast:
        print("Last error:", total_err)
        
    #gfu_new = u_grid  
    return u_grid  

In [ ]:
ua = fes.TrialFunction()
va = fes.TestFunction()

Aa = BilinearForm(fes)
Fa = LinearForm(fes)

Aa += ua*va*dx

Fa += va*dx

Aa.Assemble()
Fa.Assemble()


c = Preconditioner(Aa,"local",test = True)
c.Update()
cdir = Preconditioner(Aa,"direct",test = True)
cdir.Update()

SolveCG(Aa.mat,c.mat,gfu_a.vec,Fa.vec, fes=fes,maxsteps=10000, tol=1e-8)
SolveCG(Aa.mat,cdir.mat,gfu_a_dir.vec,Fa.vec, fes=fes, maxsteps=200, tol=1e-8)

#gfu_a.vec.data = Aa.mat.Inverse(fes.FreeDofs()) * Fa.vec

Draw(gfu_a)
Draw(gfu_a_dir)

In [ ]:
ub = fes.TrialFunction()
vb = fes.TestFunction()

Ab = BilinearForm(fes)
Fb = LinearForm(fes)

c = Preconditioner(Ab,"bddc",test = True)
#c.Update()

Ab += grad(ub)*grad(vb)*dx
Fb += vb*dx

Ab.Assemble()
Fb.Assemble()



cdir = Preconditioner(Ab, "direct",test = True)
cdir.Update()


SolveCG(Ab.mat, c.mat, gfu_b.vec, Fb.vec, fes=fes, maxsteps=1000, tol=1e-8, errConv = False)
SolveCG(Ab.mat, cdir.mat, gfu_b_dir.vec, Fb.vec, fes=fes, maxsteps=200, tol=1e-8)

#gfu_a.vec.data = Aa.mat.Inverse(fes.FreeDofs()) * Fa.vec

Draw(gfu_b)
Draw(gfu_b_dir)